In [78]:
import numpy as np, torch as t, torch.nn as nn , torch.nn.functional as F
from torchvision import datasets, models
import torch.optim as optim
import pandas as pd
from torchvision.transforms import v2 as T
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [79]:
class dataset_load():
  def __init__(self,location):
    self.loc = location
  def load_and_process(self):
    df1 = pd.read_csv("/content/MBA_ADMISSIONS.csv")
    target = df1.iloc[:,-1]
    df1 = df1.iloc[:, :7]
    df1['Target'] = target
    # d1=dict(enumerate(list(train.department.unique())))
    # d_swap = {v: k for k, v in d1.items()}
    # train['department'].replace(d_swap, inplace=True)
    df1['Target'] = df1['Target'].map({'Marketing':1, 'LOS':0, 'Finance':2, 'HR':3})
    df1.sample(7)
    return df1

d = dataset_load('/content/MBA_ADMISSIONS.csv')
df1 = d.load_and_process()

In [80]:
df1.sample(7)

,pre_score,Age_in_years,Percentage_in_10_Class,Percentage_in_12_Class,Percentage_in_Under_Graduate,percentage_MBA,post_score,Target
221,40.000000,23,93.4,96.00,83.4,67.00,90.000000,2
333,75.000000,20,88.0,79.00,69.0,57.00,55.000000,1
424,60.000000,23,90.0,92.00,79.0,72.54,75.000000,0
398,44.000000,21,97.2,91.00,68.0,62.00,66.666667,0
205,66.666667,22,82.0,78.83,65.0,69.30,63.333333,1
202,42.000000,21,89.4,95.60,77.0,65.11,86.666667,1
420,60.000000,20,93.0,92.00,77.0,69.00,85.000000,0


In [81]:
class classify(nn.Module):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(7,14)
    self.l2 = nn.Linear(14,8)
    self.l3 = nn.Linear(8,4)
    self.softmax = nn.Softmax(dim=1)
  def forward(self,x):
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = self.softmax(self.l3(x))
    return x

model = classify()

In [82]:
loss_fxn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [83]:
#calculating mean and sd for all columns.

ml =[]
sdl = []

for i in df1.columns:
  ml.append(np.mean(df1[i]))
  sdl.append(np.std(df1[i]))
  print(f'Mean of {i[:13]} is --{ml[-1]}-- and sd is --{sdl[-1]}--',end='\n')

Mean of pre_score is --68.1818181822833-- and sd is --13.829843440630812--
Mean of Age_in_years is --22.560253699788582-- and sd is --1.7059358932236102--
Mean of Percentage_in is --84.01854122621563-- and sd is --8.217045231277908--
Mean of Percentage_in is --81.55610993657504-- and sd is --9.762943686921622--
Mean of Percentage_in is --73.40513742071883-- and sd is --7.642432419706675--
Mean of percentage_MB is --67.87416490486258-- and sd is --3.756761696778134--
Mean of post_score is --76.21564482046513-- and sd is --13.501999069432133--
Mean of Target is --1.285412262156448-- and sd is --0.9224351270713493--


In [84]:
data_transformers = {
    'train': T.Compose([
        T.ToTensor(),
        T.ToDtype(t.float32, scale=True)
    ]),
    'val' : T.Compose([
        T.ToTensor(),
        T.ToDtype(t.float32, scale=True)
    ])
    }

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [85]:
from torch.utils.data import SubsetRandomSampler

#To Tensors
features = t.tensor(df1.iloc[:, :-1].values, dtype=t.float32)
targets = t.tensor(df1.iloc[:, -1].values, dtype=t.long)

# Create TensorDataset
dataset = TensorDataset(features, targets)

#Create ratio parttiiton
train_size = int(0.7 * len(df1))
test_size = len(df1) - train_size
train_sampler = SubsetRandomSampler(range(train_size))
test_sampler = SubsetRandomSampler(range(train_size, len(df1)))

# Create DataLoader
batch_size = 32
train_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
test_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)

# Print data from DataLoader
for batch_idx, (features, target) in enumerate(train_dataloader):
    print(f"Batch {batch_idx} - Features: {features}, Target: {target}")

Batch 0 - Features: tensor([[ 81.6667,  21.0000,  84.0000,  85.0000,  83.0000,  66.6000,  93.3333],
        [ 61.6667,  26.0000,  88.0000,  84.0000,  74.0000,  66.0000,  66.6667],
        [ 88.3333,  23.0000,  77.0000,  87.0000,  65.0000,  65.0000,  35.0000],
        [ 40.0000,  22.0000,  85.0000,  89.0000,  63.0000,  70.0000,  78.3333],
        [ 43.0000,  22.0000,  78.0000,  80.0000,  87.0000,  67.2700, 100.0000],
        [ 76.6667,  24.0000,  89.4000,  90.7500,  76.7000,  77.4100,  85.0000],
        [ 78.3333,  23.0000,  95.0000,  93.0000,  83.0000,  71.5000,  83.3333],
        [100.0000,  22.0000,  78.0000,  80.0000,  87.0000,  67.2700, 100.0000],
        [ 93.3333,  22.0000,  92.0000,  94.0000,  78.7100,  66.2900,  73.3333],
        [ 71.6667,  20.0000,  79.0000,  81.0000,  76.0000,  63.0000, 100.0000],
        [ 76.6667,  26.0000,  93.2000,  83.8000,  77.0000,  74.9700,  75.0000],
        [ 60.0000,  23.0000,  90.0000,  92.0000,  79.0000,  72.5400,  75.0000],
        [ 75.0000,  

In [86]:
# prompt: how to access the dataloaders during the training phase for the model neural network

# Access the dataloaders during training:

# Iterate over the training dataloader
for epoch in range(100):
  for i, data in enumerate(train_dataloader):
    # Get inputs and labels
    inputs, labels = data

    # Forward pass
    outputs = model(inputs)

    # Calculate loss
    loss = loss_fxn(outputs, labels)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Evaluate the model on the test dataloader
  correct = 0
  total = 0
  with t.no_grad():
    for i, data in enumerate(test_dataloader):
      inputs, labels = data
      outputs = model(inputs)
      predictions = t.argmax(outputs, dim=1)
      correct += t.sum(predictions == labels)
      total += len(labels)

  # Calculate accuracy
  accuracy = correct / total
  print(f"Epoch {epoch+1}, Accuracy: {accuracy}")


Epoch 1, Accuracy: 0.19637462496757507
Epoch 2, Accuracy: 0.365558922290802
Epoch 3, Accuracy: 0.365558922290802
Epoch 4, Accuracy: 0.365558922290802
Epoch 5, Accuracy: 0.3262839913368225
Epoch 6, Accuracy: 0.365558922290802
Epoch 7, Accuracy: 0.365558922290802
Epoch 8, Accuracy: 0.365558922290802
Epoch 9, Accuracy: 0.37764349579811096
Epoch 10, Accuracy: 0.365558922290802
Epoch 11, Accuracy: 0.39274924993515015
Epoch 12, Accuracy: 0.3413897156715393
Epoch 13, Accuracy: 0.365558922290802
Epoch 14, Accuracy: 0.365558922290802
Epoch 15, Accuracy: 0.365558922290802
Epoch 16, Accuracy: 0.365558922290802
Epoch 17, Accuracy: 0.365558922290802
Epoch 18, Accuracy: 0.365558922290802
Epoch 19, Accuracy: 0.3262839913368225
Epoch 20, Accuracy: 0.395770400762558
Epoch 21, Accuracy: 0.365558922290802
Epoch 22, Accuracy: 0.3806646466255188
Epoch 23, Accuracy: 0.37764349579811096
Epoch 24, Accuracy: 0.39274924993515015
Epoch 25, Accuracy: 0.3987915515899658
Epoch 26, Accuracy: 0.3897280991077423
Epoch